<a href="https://colab.research.google.com/github/31indianaSKKU/NBA_CareerLength/blob/main/CareerLength_Stathead_com_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* 문제상황: 링크로 연결된 url로 가서 데이터를 가져 오고 있지만 다시 처음의 페이지로 돌아와 다음 행의 선수 링크 url로 이동하지 못하고 있음. (에러 첨부)
-> 여러가지 문제가 복합적으로 있어서 좀 많이 고쳤습니다.
-> 일단 매번 뒤로가기 하는건 속도측면에서 비효율적이라. url먼저 수집해두고 도는 방식으로 수정 했습니다.
-> 선수 페이지로 넘어가면 도메인이 변경되면서 로그인이 풀리는 이슈가 있어 로그인 로직을 추가 했습니다.
-> 페이지 넘기는 기능은 offset값만 바꾸는 방식으로 변경 했습니다.
-> 안타깝게도... colab에서 undetected_chromedriver를 정상동작 시키는 방법을 찾지 못해서 주피터에서만 동작하는 코드를 공유 드립니다.



In [ ]:
!pip install selenium
!pip install undetected_chromedriver
!pip install webdriver_manager
!pip install pandas

In [ ]:
#수정 후 코드
#쥬피터 OK
#draft list의 선수별 데이터 가져오기(career length)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
import undetected_chromedriver as uc
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# # sheet 생성
# wb = openpyxl.Workbook()
# sheet = wb.active
# sheet.append(["no", "displayName", "team", "officialPosition", "averageSalary"])


def init_driver():
    browser = uc.Chrome()
    browser.get('https://velog.io')
    return browser

# 로그인 버튼을 눌러주고 로그인이 되서 다시 velog로 돌아올때까지 60초를 기다린다.
def do_login(browser):
    browser.find_element(By.XPATH,'//button[text()="로그인"]').click()
  
    google_login = browser.find_element(By.XPATH, '//*[@id="root"]/div[4]/div/div[2]/div[2]/div/div[1]/section[2]/div/a[2]').get_attribute('href')         
    browser.get(google_login)
    email_input = browser.find_element(By.XPATH, '//*[@id="identifierId"]')
    email_input.send_keys('jungmu.test')
    browser.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button').click()
    time.sleep(5)
    password_input = browser.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
    password_input.send_keys('quf25468!')     
    browser.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button').click()
    time.sleep(5)
    
if  __name__  ==  "__main__" :
    browser = init_driver()
    do_login(browser)
    url = 'https://stathead.com/basketball/draft_finder.cgi?request=1&order_by_asc=1&is_active=N&order_by=year_id&year_max=2020&year_min=2000&offset='

    loop_count = 0
    
    # Load the page
    browser.get(f'{url}{loop_count * 100}')
    target_element_login = browser.find_element(By.XPATH, '//*[@id="subnav"]/li[12]/a')
    target_element_login.click()
    target_element_login2 = browser.find_element(By.XPATH, '//*[@id="sh-login-google"]')
    target_element_login2.click()
    browser.get(f'{url}{loop_count * 100}')
    
    is_complete_login = False
    
    while True:
        print("loop")
        loop_count += 1
        print(loop_count)
        
        WebDriverWait(browser, 5).until(EC.presence_of_element_located((By.ID, 'stats')))  # Wait for the table to load

        # Extract the data from the table
        table = browser.find_element(By.XPATH, '//*[@id="stats"]')
        trs = table.find_elements(By.XPATH, './/tr')
        
#         rows = table.find_elements(By.XPATH, './/tr')[0:]  # Exclude header row   
        
        data_list = []

        link_list = []
        
        for tr in trs:
            try:                  
                url_cell = tr.find_elements(By.XPATH, './/td')[4]
                print(url_cell.text)
                player_url = url_cell.find_element(By.TAG_NAME, 'a').get_attribute('href')
                link_list.append(player_url.strip())
                
            except Exception as e:
                continue
                
        if(is_complete_login == False):
            new_login_url = 'https://www.basketball-reference.com/'
            browser.get(link)
            time.sleep(3)
            target_element_login = browser.find_element(By.XPATH, '//*[@id="subnav"]/li[12]/a')
            target_element_login.click()
            time.sleep(3)
            is_complete_login = True
                
        for link in link_list:
            print(link)
            browser.get(link)
            time.sleep(5)
            try:
                more_bio_button = browser.find_element(By.XPATH, '//*[@id="meta_more_button"]')
                more_bio_button.click()
            except Exception as e:
                pass  # Continue without clicking if "More Bio" button is not found             
            # Extract the data from the player page
            data_element = browser.find_element(By.CLASS_NAME, 'stats_pullout')
            data = data_element.text.strip()
            data_list.append(data)
            print(data)

        browser.get(f'{url}{loop_count * 100}')
        time.sleep(5)
        
        df = pd.DataFrame(data_list)
        
        # Print the DataFrame
        print(df)
        
        try:
            next_button = browser.find_element(By.CLASS_NAME, 'next')
        except:
            break


    # Step 4: Process is complete
    print("Crawling complete!")

    # Quit the WebDriver
    browser.quit()
    

질문3.

1. 해당 웹페이지는 구글계정 로그인이 필요한 페이지입니다.(유료사이트)

2. 목적: 첫번째 접속 페이지 내의 테이블안에 있는 player에 링크로 연결된 url로 이동 후 데이터 수집

3. 문제상황: 링크로 연결된 url로 가서 데이터를 가져 오고 있지만 다시 처음의 페이지로 돌아와 다음 행의 선수 링크 url로 이동하지 못하고 있음. (에러 첨부)


In [ ]:
#수정 전 코드
#draft list의 선수별 데이터 가져오기(career length)

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import StaleElementReferenceException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import StaleElementReferenceException
import undetected_chromedriver as uc
from selenium.common.exceptions import NoSuchElementException
import pandas as pd

# # sheet 생성
# wb = openpyxl.Workbook()
# sheet = wb.active
# sheet.append(["no", "displayName", "team", "officialPosition", "averageSalary"])


def init_driver():
    browser = uc.Chrome()
    browser.get('https://velog.io')
    return browser

# 로그인 버튼을 눌러주고 로그인이 되서 다시 velog로 돌아올때까지 60초를 기다린다.
def do_login(browser):
    browser.find_element(By.XPATH,'//button[text()="로그인"]').click()
  
    google_login = browser.find_element(By.XPATH, '//*[@id="root"]/div[4]/div/div[2]/div[2]/div/div[1]/section[2]/div/a[2]').get_attribute('href')         
    browser.get(google_login)
    email_input = browser.find_element(By.XPATH, '//*[@id="identifierId"]')
    email_input.send_keys('구글계정')
    browser.find_element(By.XPATH, '//*[@id="identifierNext"]/div/button').click()
    time.sleep(7)
    password_input = browser.find_element(By.XPATH, '//*[@id="password"]/div[1]/div/div[1]/input')
    password_input.send_keys('구글계정비번')     
    browser.find_element(By.XPATH, '//*[@id="passwordNext"]/div/button').click()
    time.sleep(15)
    
if  __name__  ==  "__main__" :
    browser = init_driver()
    do_login(browser)
    url = 'https://stathead.com/basketball/draft_finder.cgi?request=1&order_by_asc=1&is_active=N&order_by=year_id&year_max=2020&year_min=2000&offset=0'

    while True:
        # Load the page
        browser.get(url)
        target_element_login = browser.find_element(By.XPATH, '//*[@id="subnav"]/li[12]/a')
        target_element_login.click()
        target_element_login2 = browser.find_element(By.XPATH, '//*[@id="sh-login-google"]')
        target_element_login2.click()
        browser.get(url)
        WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'stats')))  # Wait for the table to load

        # Extract the data from the table
        table = browser.find_element(By.XPATH, '//*[@id="stats"]')
        trs = table.find_elements(By.XPATH, './/tr')
        
#         rows = table.find_elements(By.XPATH, './/tr')[0:]  # Exclude header row   
        
        data_list = []


        for tr in trs:        
            try:
                url_cell = tr.find_element(By.XPATH, '//*[@id="stats"]/tbody/tr[1]/td[5]')
                player_url = url_cell.find_element(By.TAG_NAME, 'a').get_attribute('href')
#             except StaleElementReferenceException:
                
                browser.get(player_url)
                try:
                    more_bio_button = browser.find_element(By.XPATH, '//*[@id="meta_more_button"]')
                    more_bio_button.click()
                except NoSuchElementException:
                    pass  # Continue without clicking if "More Bio" button is not found             
             # Extract the data from the player page
                data_element = browser.find_element(By.CSS_SELECTOR, '#meta div:nth-of-type(2)')
                data = data_element.text.strip()
                data_list.append(data)
                print(data_list)
                time.sleep(5)
############################################################################################################################################################################### 
###############################################################################################################################################################################
###############################################################################################################################################################################
###############################################################################################################################################################################  
#문제 부분                 
                # Go back to the previous page
                browser.back()   #//*[@id="stats"]/tbody/tr[2]/td[5]
                time.sleep(5)
                continue               
                
            except NoSuchElementException:
                continue
        
        
        df = pd.DataFrame(data_list)
        
        # Print the DataFrame
        print(df)

        # Check if there is a next page
        next_button = browser.find_element(By.CSS_SELECTOR, '#stathead_results div:last-of-type a')
        if not next_button:
            break

        # Get the URL of the next page
        next_url = next_button.get_attribute('href')

        # Build the full URL for the next page
        url = f'https://stathead.com{next_url}'

    # Step 4: Process is complete
    print("Crawling complete!")

    # Quit the WebDriver
    driver.quit()
    